<a href="https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemyst_recurrent_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Demystified | Recurrent Neural Networks
https://nlpdemystified.org<br>
https://github.com/nitinpunjabi/nlp-demystified

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on the free tier, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this for free in the cloud rather than using a paid tier or using a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity. If that happens and you don't reconnect in time, you will need to upgrade spaCy again and reinstall the requisite statistical package(s).
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

# Part-of-Speech Tagging with a Bidirectional LSTM

It's difficult to find free sequence labelling datasets because they're so labour-intensive to create.
<br><br>
Fortunately, **Natural Language Toolkit (NLTK)** includes enough free sets of corpora for our purposes. NLTK also provides them in a convenient uniform format.<br>
https://www.nltk.org/index.html<br>
https://www.nltk.org/nltk_data/<br>
<br>
We'll use the Treebank, Brown, and CONLL-2000 datasets. 

In [ ]:
import nltk

nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')

In their original form, not all corpora use the same tagsets, so we'll also download the *universal_tagset* from NLTK so that all datasets share the same set.<br>
https://universaldependencies.org/u/pos/

In [ ]:
nltk.download('universal_tagset')

In [ ]:
from nltk.corpus import treebank, brown, conll2000

In [ ]:
# Download all PoS-tagged sentences and place them in one list.
tagged_sentences = treebank.tagged_sents(tagset='universal') +\
                   brown.tagged_sents(tagset='universal') +\
                   conll2000.tagged_sents(tagset='universal')

Each tagged sentence is a list of (token, tag) tuples.

In [ ]:
print(tagged_sentences[0])
print(len(tagged_sentences))

We need to separate the tokens from the tags. Each sequence of tokens (sentence) will be an input to our model, and each corresponding sequence of tags will be a label for that sentence.

In [ ]:
sentences, sentence_tags = [], []

for s in tagged_sentences:
  sentence, tags = zip(*s)
  sentences.append(list(sentence))
  sentence_tags.append(list(tags))

In [ ]:
print(sentences[0])
print(sentence_tags[0])

In [ ]:
print(len(sentences), len(sentence_tags))

Create train/validation/test splits. This time, we don't have a separate test set so we'll call *train_test_split* twice.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

x_train, x_test, y_train, y_test = train_test_split(sentences, sentence_tags, test_size=1 - train_ratio)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

In [ ]:
print(len(x_train), len(y_train))
print(len(x_val), len(y_val))
print(len(x_test), len(y_test))

If you watched the demo on **Static Word Embeddings**, the next few steps should look familiar.<br>
https://github.com/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemyst_static_word_embeddings.ipynb

First, we need to create a tokenizer for the sentences and fit it to create a vocabulary.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
from tensorflow import keras

In [ ]:
sentence_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<OOV>')

In [ ]:
sentence_tokenizer.fit_on_texts(x_train)

In [ ]:
print(len(sentence_tokenizer.word_index))
sentence_tokenizer.get_config()

We also need to create another tokenizer for the tags since our labels are also sequences.

In [ ]:
tag_tokenizer = keras.preprocessing.text.Tokenizer()
tag_tokenizer.fit_on_texts(y_train)

In [ ]:
print(len(tag_tokenizer.word_index))
tag_tokenizer.get_config()

The **universal tagset** is a reduced tag list so items such as *proper nouns* are missing.

In [ ]:
tag_tokenizer.word_index

Next, we need to vectorize our sentences and corresponding tags.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences

In [ ]:
x_train_seqs = sentence_tokenizer.texts_to_sequences(x_train)

In [ ]:
print(x_train_seqs[0])

In [ ]:
sentence_tokenizer.sequences_to_texts([x_train_seqs[0]])

In [ ]:
y_train_seqs = tag_tokenizer.texts_to_sequences(y_train)

In [ ]:
tag_tokenizer.sequences_to_texts([y_train_seqs[0]])

In [ ]:
# Vectorize the validation sentences and tags.
x_val_seqs = sentence_tokenizer.texts_to_sequences(x_val)
y_val_seqs = tag_tokenizer.texts_to_sequences(y_val)

As we covered in the slides, **Recurrent Neural Networks** are capable of handling variable length sequences.<br><br>
Despite that, it's still best to pad sequences to a uniform length for one or both of these reasons:<br>
1. Performance. The longer a sequence, the higher the computation cost. One may want to truncate all sequences to the same length if that's feasible.
2. When processing datasets in batches, each sequence in a batch usually has to be of uniform length.<br>

For simplicity, in this demo, we'll make *every* sequence be as long as the longest sequence. A more optimized solution would be to make each sequence as long as the longest sequence in each *batch* to avoid unnecessary processing.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
MAX_LENGTH = len(max(x_train_seqs, key=len))
print(MAX_LENGTH)

In [ ]:
x_train_padded = keras.preprocessing.sequence.pad_sequences(x_train_seqs, padding='post', maxlen=MAX_LENGTH)

In [ ]:
print(x_train_padded[0])

In [ ]:
y_train_padded = keras.preprocessing.sequence.pad_sequences(y_train_seqs, padding='post', maxlen=MAX_LENGTH)

In [ ]:
# Pad the validation sentences and tags.
x_val_padded = keras.preprocessing.sequence.pad_sequences(x_val_seqs, padding='post', maxlen=MAX_LENGTH)
y_val_padded = keras.preprocessing.sequence.pad_sequences(y_val_seqs, padding='post', maxlen=MAX_LENGTH)

PoS tagging is a multiclass classification task done at each timestep, so we need to convert every tag for every sentence into a one-hot encoding.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical<br>

In [ ]:
y_train_categoricals = keras.utils.to_categorical(y_train_padded)

A sequence of tags for a single sentence is now a sequence of one-hot encodings.

In [ ]:
print(y_train_categoricals[0])

In [ ]:
# One-hot encoding for a single tag.
print(y_train_categoricals[0][0])

In [ ]:
# Retrieving the corresponding tag.
import numpy as np
print(tag_tokenizer.index_word[np.argmax(y_train_categoricals[0][0])])

In [ ]:
# Turn the validation set tags into one-hot encodings as well.
y_val_categoricals = keras.utils.to_categorical(y_val_padded)

In [ ]:
# For the embedding layer.
num_tokens = len(sentence_tokenizer.word_index) + 1
embedding_dim = 128

# For the output layer.
num_classes = len(tag_tokenizer.word_index) + 1

At this point, we're ready to build our model. We'll train word embeddings concurrently with our model (though you can use pretrained word vectors as well).<br><br>
There are several new things here:<br>
1. The embedding layer has a *mask_zero* parameter. We added padding in order to make our batches the same size, but we don't want the model to make PoS predictions on padding. Setting *mask_zero* to True makes the layers following the embedding layer ignore padding values.<br>
https://www.tensorflow.org/guide/keras/masking_and_padding<br>
https://stackoverflow.com/questions/47485216/how-does-mask-zero-in-keras-embedding-layer-work<br><br>
2. We're using a **bidirectional LSTM**. The *Bidirectional* layer is a wrapper to which we pass an *LSTM* layer. The first parameter to the *LSTM* layer is the number of units in the cell. The second parameter, *return_sequences*, controls whether the RNN returns an output for each timestep or only the last output. Since we're doing PoS-tagging, we want an output for each timestep.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM<br><br>
3. A *TimeDistributed* layer wraps the *Dense* output layer. This way, the *Dense* layer with its *softmax* activation function gets applied to **every** sequential output to produce a PoS tag prediction.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed

In [ ]:
from keras import layers

In [ ]:
model = keras.Sequential()

model.add(layers.Embedding(input_dim=num_tokens, 
                           output_dim=embedding_dim, 
                           input_length=MAX_LENGTH,
                           mask_zero=True))
model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True)))
model.add(layers.TimeDistributed(layers.Dense(num_classes, activation='softmax')))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(x_train_padded, y_train_categoricals, epochs=20, batch_size=256, validation_data=(x_val_padded, y_val_categoricals), callbacks=[es_callback])

In [ ]:
# Preprocess the test data and test the model.
x_test_seqs = sentence_tokenizer.texts_to_sequences(x_test)
x_test_padded = keras.preprocessing.sequence.pad_sequences(x_test_seqs, padding='post', maxlen=MAX_LENGTH)

y_test_seqs = tag_tokenizer.texts_to_sequences(y_test)
y_test_padded = keras.preprocessing.sequence.pad_sequences(y_test_seqs, padding='post', maxlen=MAX_LENGTH)
y_test_categoricals = keras.utils.to_categorical(y_test_padded)

In [ ]:
model.evaluate(x_test_padded, y_test_categoricals)

We can now use our model to tag sentences.

In [ ]:
samples = [
    "Brown refused to testify.",
    "Brown sofas are on sale.",
]

The function below takes a list of strings, tokenizes and pads them, then has the model tag them.

In [ ]:
def tag_sentences(sentences):
  sentences_seqs = sentence_tokenizer.texts_to_sequences(sentences)
  sentences_padded = keras.preprocessing.sequence.pad_sequences(sentences_seqs, maxlen=MAX_LENGTH, padding='post')

  # The model returns a list of PROBABILITY DISTRIBUTIONS (due to the softmax)
  # for EACH sentence. There is one probability distribution for each PoS tag.
  tag_preds = model.predict(sentences_padded)

  sentence_tags = []

  # For EACH LIST of probability distributions...
  for i, preds in enumerate(tag_preds):

    # Extract the most probable tag from EACH probability distribution.
    tags_seq = [np.argmax(p) for p in preds[:len(sentences_seqs[i])]]

    # Convert the sentence and tag sequences back to their token counterparts.
    words = [sentence_tokenizer.index_word[w] for w in sentences_seqs[i]]
    tags = [tag_tokenizer.index_word[t] for t in tags_seq]
    sentence_tags.append(list(zip(words, tags)))

  return sentence_tags


In [ ]:
tagged_sample_sentences = tag_sentences(samples)

In [ ]:
print(tagged_sample_sentences[0])

In [ ]:
print(tagged_sample_sentences[1])

# Language Modelling With Stacked LSTMs

We'll build a language model trained on the *Art of War* by Sun Tzu.

In [ ]:
import requests
art_of_war = requests.get('https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/datasets/art_of_war.txt')\
                     .text

The language model we'll build will be **character**-based (as opposed to word-based). That is, given a sequence of one or more characters, the model will be asked to predict the next character.<br><br>
Character-level models have the advantage of:
- Smaller prediction space. There are only a handful of characters in the English language compared to the tens of thousands of words in a typical corpus.
- Character-level models are more resilient to out-of-vocabulary (OOV) conditions and are better able to learn the lower mechanics of language (including punctuation).<br><br>

On the other hand, character-level models need to learn a sequence of characters to "make sense" of a word (e.g. the sequence of "c", "a", "t" to identify "cat" as a pattern) which can be inefficient.<br><br>
RNNs can process any kind of sequence so what's shown here can easily be applied at the word level.

We'll initialize a Keras **Tokenizer** and set the *char_level* parameter to True so that our corpus gets tokenized into characters rather than words.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
from tensorflow import keras
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [ ]:
tokenizer.fit_on_texts([art_of_war])

The tokenizer's internal dictionary now maps characters rather than words...

In [ ]:
tokenizer.get_config()

...and the resulting possibility space is much smaller.

In [ ]:
len(tokenizer.word_index)

As before, we'll turn the book's characters into a sequence of integers.

In [ ]:
seq = tokenizer.texts_to_sequences([art_of_war])[0]

In [ ]:
len(seq)

In [ ]:
tokenizer.sequences_to_texts([seq[:10]])

To segment the vectorized corpus into training examples, we'll use the **Tensorflow Data** API which makes it easy to build preprocessing pipelines by chaining operations together.<br>
https://www.tensorflow.org/guide/data<br>
https://www.tensorflow.org/api_docs/python/tf/data<br>

The first step is to convert the vectorized corpus into a stream of character indices using *from_tensor_slices*.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices

In [ ]:
import tensorflow as tf
slices = tf.data.Dataset.from_tensor_slices(seq)

This returns a Tensorflow **Dataset** object, an abstraction that represents a sequence of elements.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [ ]:
type(slices)

Like Python **generators**, TF **Datasets** function like iterators, so we need to do things like convert them into a list or iterate over them in order to print them.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take

In [ ]:
# Create a Dataset from the first ten slices and convert to a list to output to console.
list(slices.take(10))

In [ ]:
# The first ten elements from the original vectorized corpus.
seq[:10]

The next step is to create the training examples. To do that, we'll use the *window* method. Calling *window* on a dataset results in a sequence of datasets, each containing N elements. So calling window(100) on a dataset of 1000 will result in a sequence of 10 datasets, each containing 100 elements.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window

Here, we're creating windows of *input_timesteps* + 1. The *input_timesteps* represents our training example length. The *+1* is there to help us create the target/label for each training example. This will be clarified further below.<br><br>
In addition, we're setting *shift* to 1. This means the result will contain overlapping windows shifted by 1. e.g. if the input is [1, 2, 3, 4, ...]. The first window will contain [1, 2, 3, ...], the second window will contain [2, 3, 4, ...] and so on. This is to create more training examples.<br><br>
Finally, we're setting *drop_remainder* to True which ensures ALL windows contain exactly N elements. i.e. once the input contains fewer than N elements, they are ignored.

In [ ]:
input_timesteps = 100
window_size = input_timesteps + 1
windows = slices.window(window_size, shift=1, drop_remainder=True)

Iterating through a subset of windows, we can see they're all the same length and that each subsequent window is shifted over by 1.

In [ ]:
for w in windows.take(3):
  arr = list(w.as_numpy_iterator())
  print(len(arr), arr)

The *window* method returns a nested dataset of datasets (i.e. each window is a dataset)...

In [ ]:
for w in windows.take(2):
  print(w)

...but we need tensors for our model. So we can use the window's *batch* method to convert each window object back to a tensor, then *flat_map* to flatten the results to a single(i.e. non-nested) dataset of tensors.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#flat_map

In [ ]:
dataset = windows.flat_map(lambda window: window.batch(window_size))

We now have a single dataset of tensors, where each tensor is *window_size* long and shifted by 1.

In [ ]:
for d in dataset.take(2):
  print(d)

The next step is to create batches from our dataset. To do this, we'll shuffle the dataset to give the model optimizer a better chance of "bouncing out of a local minimum", then create batches.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle<br>


In [ ]:
batch_size = 32

In [ ]:
# The expected number of batches should be (len(seq) - input_timesteps) / batch_size
batches = dataset.shuffle(10000).batch(batch_size)

In [ ]:
for b in batches.take(2):
  print(b)

We can now separate each example into an input(x) and a corresponding label(y).<br><br>
In the slides, we talked about **Teacher Forcing** where:<br>
1. At each timestep during training, the output is compared to a label.
2. At the next timestep, rather than feeding the model the previous output, we feed it the next character of the input sequence (i.e. what the model should've outputted).
<br><br>

This is why each window is of size *input_timesteps + 1*. Each window is now going to be separated into TWO sequences. The first sequence will be the training input and will be of length *input_timesteps* (i.e. everything but the LAST character). The second sequence will be the label and will consist of all the window elements shifted by 1 (i.e. everything but the FIRST character).

In [ ]:
xy_batches = batches.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

Each batch now consists of a set of inputs and a set of labels, with the labels shifted over by 1.

In [ ]:
for b in xy_batches.take(1):
  print(b)

In [ ]:
# For greater clarity.
for b in xy_batches.take(1):
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1: ", b[1][0].numpy())

In [ ]:
num_tokens = len(tokenizer.word_index) + 1

The last step before we can build our model is to one-hot encode the inputs. We're doing this because:
1. We're not using embeddings for the input. We can, but since this is a character model with just a few dozen possible choices, we can get away with one-hot encoding.
2. Since we're not using embeddings and our input is categorical, we need to one-hot encode.

Note that despite our labels ALSO being categorical, we are NOT one-hot encoding them. This is because we'll be using a loss function that can help us skip that step (more below).

In [ ]:
xy_batches = xy_batches.map(lambda inputs, labels: (tf.one_hot(inputs, depth=num_tokens), labels))

In [ ]:
for b in xy_batches.take(1):
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1: ", b[1][0].numpy())

The last step is to add some **prefetching**. This is an optimization step. This way, while the model trains on the current batch of data, the pipeline reads and prepares the next batch.<br>
https://www.tensorflow.org/guide/data_performance#prefetching

In [ ]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

We can now build our model. There are three new things here:
1. We're stacking two LSTMs. The sequential output of the first LSTM will become the sequential input of the second LSTM.<br><br>
2. We're adding some *recurrent_dropout*. This drops connections between the recurrent units (i.e. the dropout is applied horizontally across time). You can still use regular *dropout* as well which will be applied to the inputs/outputs. Refer to this paper for more information:<br>
https://arxiv.org/abs/1512.05287<br><br>
3. We're using **sparse_categorical_crossentropy**. This allows us to provide labels as integers rather than one-hot encodings.<br>
https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class

In [ ]:
from keras import layers

In [ ]:
model = keras.models.Sequential()

model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))
model.add(layers.TimeDistributed(layers.Dense(num_tokens, activation='softmax')))

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')


In [ ]:
model.summary()

Because this model takes a few hours to train, we're using **model checkpoints** to save the weights after every epoch. This way, if something goes wrong with our system during training, we can reload the last set of weights from the checkpoint, and resume training from there.<br>
https://keras.io/api/callbacks/model_checkpoint/

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
# Saving this to a folder on my local machine (possible because I'm running this notebook on a local Jupyter backend).
filepath="./ArtofWarLM/training1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

When calling the model's *fit* method, we:<br>
1. simply pass in the batches as is (no need to separate into explicit x and y arguments).
2. pass the model checkpoint callback to save the weights after every epoch

Note the call to *fit* below is commented out. Because this model takes a few hours to train, I trained it ahead of time and saved it. If you want to train it yourself, feel free to uncomment and execute it.

**Note**: Because of the random weight initialization, your trained model's output will likely differ from mine.

In [ ]:
# history = model.fit(xy_batches, epochs=50, callbacks=[cp_callback])

Once training is complete, we can call the model's *save* method to save its weights and metadata. Again, this is commented out because I already trained the model previously.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/Model#save<br>
https://www.tensorflow.org/guide/keras/save_and_serialize

In [ ]:
# model.save('art_of_war_char_level_lm')

Download and unzip the previously trained model...

In [ ]:
!wget https://github.com/nitinpunjabi/nlp-demystified/raw/main/models/art_of_war_char_level_lm.zip
!unzip -o art_of_war_char_level_lm.zip

...and load it.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model

In [ ]:
model = keras.models.load_model('art_of_war_char_level_lm')

In [ ]:
import numpy as np

Now that we have a trained model, let's generate some text.<br><br>
The method below takes some seed text and uses that to generate a certain number of characters. For each character, it uses the currently generated text so far as the input.<br><br>
There's also a *temperature* parameter. The next character is picked from a probability distribution. By dividing the log of this distribution by *temperature*, we can influence the randomness of the output.<br><br>
When the temperature is low (< 1), the probability distribution sharpens and the model will be more strict in recreating the original text. As we raise the temperature, the distribution flattens and there's a higher chance the model picks something unexpected, resulting in greater surprise in the output. In practice, a high enough temperature will result in nonsense.

In [ ]:
def generate_text(model, tokenizer, seed_text, num_chars=200, temperature=1):

  text = seed_text

  for _ in range(num_chars):
    
    # Create input sequence of length time_steps.
    input = np.array(tokenizer.texts_to_sequences([text[-input_timesteps:]]))
    input = tf.one_hot(input, num_tokens)

    # Create probability distribution for next character adjusted by temperature.
    preds = model.predict(input)[0, -1:, :] # <-- We want only the last character predicted.
    preds = tf.math.log(preds) / temperature

    # Choose next character and add to running text.
    next_char = tf.random.categorical(preds, num_samples=1)
    next_char = tokenizer.sequences_to_texts(next_char.numpy())[0]

    text += next_char
  
  return text


In [ ]:
print(generate_text(model, tokenizer, "Banana peels on the battlefield can", num_chars=300, temperature=0.2))

In [ ]:
print(generate_text(model, tokenizer, "It's time to release the Kraken when", num_chars=300, temperature=0.5))

In [ ]:
print(generate_text(model, tokenizer, "Crush your enemies, see them driven before you, and", num_chars=300, temperature=1))

In [ ]:
print(generate_text(model, tokenizer, "What is best in life?", num_chars=300, temperature=2))

A few observations of the preceding outputs:
1. Despite being a character-level model, the model managed to "learn" spelling, cadence, punctuation, spacing, grammar, and even numbered bullet points just from trying to predict the next character.
2. It's pretty cool how the model manages to take our initial seed text and complete a sentence with it before moving on.
3. We can see the output getting increasingly nonsensical as the temperature rises. What temperature to use ultimately depends on the nature of your corpus and your goals with the language model.

# Further Exploration
1. Check out *Sunspring*, a sci-fi short written by an LSTM. The director and actors played the script straight and the result is hilarious.<br>
https://www.youtube.com/watch?v=LY7x2Ihqjmc<br>
https://en.wikipedia.org/wiki/Sunspring<br><br>
2. Everything we learned here can be applied at the word-level. Try creating a word-level language model with a different corpus (maybe download something from https://www.gutenberg.org/) and try using word embeddings.<br><br>
3. We didn't evaluate our language model using perplexity. Find out online how to do it.